In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/home/shaoz/projects/collaborate/spgc/AD/MADReSS-23-train/training-groundtruth.csv')

In [3]:
df.head(5)

,adressfname,age,gender,educ,dx,mmse
0,adrso002,70,female,NaN,Control,26.0
1,adrso003,72,female,NaN,Control,30.0
2,adrso004,74,female,NaN,Control,30.0
3,adrso005,67,female,NaN,Control,27.0
4,adrso006,65,female,NaN,Control,28.0


In [4]:
df.shape

(237, 6)

In [10]:
list(df['dx'].unique())

['Control', 'ProbableAD']

In [7]:
item = df.loc[2,:]

In [8]:
item['dx']

'Control'

In [1]:
import torch
torch.distributed.is_available()

/home/shaoz/mambaforge/envs/s3prl/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
torch.distributed.is_initialized()

False

In [3]:
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
available_gpus

In [5]:
torch.nn.DataParallel

torch.nn.parallel.data_parallel.DataParallel

In [6]:
help(torch.nn.DataParallel)

Help on class DataParallel in module torch.nn.parallel.data_parallel:

class DataParallel(torch.nn.modules.module.Module)
 |  DataParallel(module, device_ids=None, output_device=None, dim=0)
 |  
 |  Implements data parallelism at the module level.
 |  
 |  This container parallelizes the application of the given :attr:`module` by
 |  splitting the input across the specified devices by chunking in the batch
 |  dimension (other objects will be copied once per device). In the forward
 |  pass, the module is replicated on each device, and each replica handles a
 |  portion of the input. During the backwards pass, gradients from each replica
 |  are summed into the original module.
 |  
 |  The batch size should be larger than the number of GPUs used.
 |  
 |  .. warning::
 |      It is recommended to use :class:`~torch.nn.parallel.DistributedDataParallel`,
 |      instead of this class, to do multi-GPU training, even if there is only a single
 |      node. See: :ref:`cuda-nn-ddp-instead`

In [7]:
help(torch.distributed.init_process_group)

Help on function init_process_group in module torch.distributed.distributed_c10d:

init_process_group(backend: Union[str, torch.distributed.distributed_c10d.Backend], init_method: Optional[str] = None, timeout: datetime.timedelta = datetime.timedelta(seconds=1800), world_size: int = -1, rank: int = -1, store: Optional[torch.distributed.distributed_c10d.Store] = None, group_name: str = '', pg_options: Optional[Any] = None)
    Initializes the default distributed process group, and this will also
    initialize the distributed package.
    
    There are 2 main ways to initialize a process group:
        1. Specify ``store``, ``rank``, and ``world_size`` explicitly.
        2. Specify ``init_method`` (a URL string) which indicates where/how
           to discover peers. Optionally specify ``rank`` and ``world_size``,
           or encode all required parameters in the URL and omit them.
    
    If neither is specified, ``init_method`` is assumed to be "env://".
    
    
    Args:
     

In [5]:
import torch
import torch.distributed as dist
import os

os.environ['MASTER_ADDR'] = "127.0.0.1"
os.environ['MASTER_PORT'] = "12355"
os.environ['RANK'] = "0"
os.environ['WORLD_SIZE'] = "2"

def get_mpi_rank():
    return int(os.environ['RANK'])

def get_mpi_size():
    return int(os.environ.get('WORLD_SIZE', '1'))

rank = get_mpi_rank()
world_size = get_mpi_size()

init_param={'backend': 'nccl',
        'init_method': 'env://',
        'rank': rank,
        'world_size': world_size}

from pprint import pformat
print('before {} - {}\n'.format(rank, pformat(init_param)))
dist.init_process_group(**init_param)
print('after {}'.format(rank))

before 0 - {'backend': 'nccl', 'init_method': 'env://', 'rank': 0, 'world_size': 2}



KeyboardInterrupt: 